In [1]:
import numpy as np

In [2]:
# Task durations and available time at each facility
task_times = [5, 8, 4, 7, 6, 3, 9]
facility_limits = [24, 30, 28]

# Cost of assigning each task to each facility
costs = [
    [10, 12, 9],
    [15, 14, 16],
    [8, 9, 7],
    [12, 10, 13],
    [14, 13, 12],
    [9, 8, 10],
    [11, 12, 13]
]


# GA Parameters
population_size = 6
max_generations = 100
crossover_chance = 0.8
mutation_chance = 0.2

In [3]:
def calculate_fitness(individual):
    used_time = [0, 0, 0]
    cost = 0
    penalty = 0

    for i in range(len(individual)):
        facility = individual[i] - 1
        used_time[facility] += task_times[i]
        cost += task_times[i] * costs[i][facility]

    for f in range(3):
        if used_time[f] > facility_limits[f]:
            penalty += (used_time[f] - facility_limits[f]) * 1000

    return 1 / (cost + penalty)

# Roulette wheel selection
def select(pop, scores):
    total = sum(scores)
    prob = [s / total for s in scores]
    chosen = np.random.choice(len(pop), p=prob)
    return pop[chosen]

# Crossover
def do_crossover(p1, p2):
    if np.random.rand() < crossover_chance:
        point = np.random.randint(1, len(p1))
        c1 = p1[:point] + p2[point:]
        c2 = p2[:point] + p1[point:]
        return c1, c2
    return p1[:], p2[:]

# Mutation
def do_mutation(ind):
    if np.random.rand() < mutation_chance:
        i, j = np.random.choice(len(ind), 2, replace=False)
        ind[i], ind[j] = ind[j], ind[i]
    return ind


In [4]:
# Generate initial population
population = []
for _ in range(population_size):
    individual = list(np.random.randint(1, 4, len(task_times)))
    population.append(individual)

# Main GA loop
for generation in range(max_generations):
    scores = [calculate_fitness(ind) for ind in population]
    next_gen = []

    for _ in range(population_size // 2):
        parent1 = select(population, scores)
        parent2 = select(population, scores)
        child1, child2 = do_crossover(parent1, parent2)
        next_gen.append(do_mutation(child1))
        next_gen.append(do_mutation(child2))

    population = next_gen

# Get best solution
final_scores = [calculate_fitness(ind) for ind in population]
best = population[np.argmax(final_scores)]

# Print result
print("Best task assignment to facilities:", best)
total_cost = sum(task_times[i] * costs[i][best[i] - 1] for i in range(len(task_times)))
print("Final total cost:", total_cost)

Best task assignment to facilities: [3, 1, 3, 2, 1, 3, 1]
Final total cost: 476
